In [4]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import polars as pl
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x :.1f}')
from cleaner import DataCleaner
import numpy as np
from datetime import datetime
from tools import (
    order_and_rename,
    import_datasets,
    bins_generator
)


In [5]:
movies = pd.read_csv('clean_datasets/movies_rating.csv', index_col=0)
ratings = pd.read_csv(
    'movies_datasets/title_ratings.tsv',
    sep = "\t",
)
joined = pd.merge(
    movies,
    ratings,
    left_on = "titre_id",
    right_on = "tconst"
)

In [6]:
joined = joined.rename(
    columns={
        "averageRating" : "rating_avg",
        "numVotes" : "rating_votes"
    }
).drop("tconst", axis=1)

rating = joined

In [7]:
bins, names = bins_generator(rating["titre_date_sortie"].max())

rating["cuts"] = pd.cut(
    rating["titre_date_sortie"],
    bins=bins,
    labels=names
)

In [8]:
rating["rating_avg"].median()

6.1

In [9]:
rating["rating_avg"].describe()

count   1053776.0
mean          6.0
std           1.3
min           1.0
25%           5.2
50%           6.1
75%           6.9
max          10.0
Name: rating_avg, dtype: float64

In [10]:
condi = (
    rating["rating_avg"] == 10
)

best_movies = rating[condi]

best_movies

,titre_id,titre_str,titre_type,titre_date_sortie,titre_date_fin,titre_duree,titre_genres,person_id,person_name,person_birthdate,person_job,person_role,person_index,rating_avg,rating_votes,cuts
565926,tt10449358,Kaputol,movie,2019,0,120,Drama,nm0317737,Cherie Gil,1963,actress,"[""Kiki"",""Rina""]",1,10.0,7,2010-2020
565927,tt10449358,Kaputol,movie,2019,0,120,Drama,nm1368756,Alfred Vargas,1981,actor,"[""Caloy"",""Robert""]",2,10.0,7,2010-2020
565928,tt10449358,Kaputol,movie,2019,0,120,Drama,nm6651713,Ronwaldo Martin,0,actor,"[""Conrad""]",3,10.0,7,2010-2020
565929,tt10449358,Kaputol,movie,2019,0,120,Drama,nm0032629,Angel Aquino,1973,actress,"[""Maybelle""]",4,10.0,7,2010-2020
566407,tt10463270,Shouting Silence,movie,2021,0,95,"Crime,Drama,Thriller",nm12297451,Samir Singh,0,actor,"[""Tushar""]",10,10.0,18,2010-2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1036409,tt8974670,Major Capers: The Legend of Team Broadminded,movie,2022,0,88,Documentary,nm10109594,Doc Lowell Burwell,0,actor,0,3,10.0,6,>2021
1039230,tt9080346,Poets Are the Destroyers,movie,2021,0,0,Drama,nm9684744,Aarushi Agni,0,actress,"[""Jazz Singer""]",1,10.0,6,2010-2020
1039231,tt9080346,Poets Are the Destroyers,movie,2021,0,0,Drama,nm10181391,Pablo Javier Alfaro,0,actor,"[""Jazz Musician""]",2,10.0,6,2010-2020
1039232,tt9080346,Poets Are the Destroyers,movie,2021,0,0,Drama,nm10169618,Shepsi Haider,0,actor,"[""Pablo""]",3,10.0,6,2010-2020


In [12]:
for date in names[1:]:
    condi = (
        (rating["cuts"] == date)
    )


    print(f"Average note for {date}, {rating['rating_avg'][condi].median()}")

Average note for 1900-1920, 6.0
Average note for 1920-1940, 6.2
Average note for 1940-1960, 6.3
Average note for 1960-1980, 6.1
Average note for 1980-1990, 6.0
Average note for 1990-2000, 6.0
Average note for 2000-2010, 6.1
Average note for 2010-2020, 6.0
Average note for >2021, 6.4


In [13]:
condi = rating["rating_avg"] > 9

rating["titre_genres"][condi].mode()

0    Drama
Name: titre_genres, dtype: object

Avg rating par genre, avg rating par acteur, avg par director, correlation entre durée et note?